# Solver for Qodec's lunchbox sudoku problem

https://logic-masters.de/Raetselportal/Raetsel/zeigen.php?id=000451

In [224]:
from ortools.sat.python import cp_model

## Define the Sudoku board

In [225]:
model = cp_model.CpModel()
def create_row_vars(y):
    return [model.NewIntVar(1, 9, 'x:{},y:{}'.format(x,y)) for x in range(9)] 
nodes = [create_row_vars(y) for y in range(9)]

## Add the normal sudoku constraints

In [226]:
for x in range(9):
    model.AddAllDifferent(nodes[x])

In [227]:
for y in range(9):
    model.AddAllDifferent([nodes[x][y] for x in range(9)])

In [228]:
for x_offset in range(0, 9, 3):
    for y_offset in range(0, 9, 3):
        square = []
        for x in range(3):
            for y in range(3):
                square.append(nodes[x_offset+x][y_offset+y])
        model.AddAllDifferent(square)

## Defining sandwich constraints

In [229]:
def vertical_sandwich(x, top, bottom, model, nodes):
    b = model.NewBoolVar('vsandwich{},{}:{}_b'.format(y,top,bottom))
    model.Add(sum(nodes[x][top+1:bottom]) == nodes[x][top]*10+ nodes[x][bottom])\
                    .OnlyEnforceIf(b)    
    model.Add(sum(nodes[x][top+1:bottom]) == nodes[x][top]   + 10*nodes[x][bottom])\
                    .OnlyEnforceIf(b.Not())    

In [231]:
def horizontal_sandwich(y, left, right, model, nodes):
    b = model.NewBoolVar('hsandwich{},{}:{}_b'.format(x,left,right))
    model.Add(sum([nodes[x][y] for x in range(left+1,right)]) == nodes[left][y]*10+ nodes[right][y])\
                    .OnlyEnforceIf(b)
    model.Add(sum([nodes[x][y] for x in range(left+1,right)]) == nodes[left][y]   + 10*nodes[right][y])\
                    .OnlyEnforceIf(b.Not())

## Defining the puzzle

In [232]:
model.Add(nodes[0][7]==6)

In [233]:
horizontal_sandwich(1,3,7,model,nodes)
horizontal_sandwich(3,0,5,model,nodes)
horizontal_sandwich(4,0,5,model,nodes)
horizontal_sandwich(5,0,5,model,nodes)
horizontal_sandwich(8,4,7,model,nodes)

In [234]:
vertical_sandwich(2,1,8, model, nodes)
vertical_sandwich(3,0,7, model, nodes)
vertical_sandwich(3,2,5, model, nodes)
vertical_sandwich(6,3,6, model, nodes)
vertical_sandwich(7,0,7, model, nodes)
vertical_sandwich(8,1,8, model, nodes)

## Solving it

In [235]:
solver = cp_model.CpSolver()
status = solver.Solve(model)
status

4

In [236]:
def print_solution(solver, nodes):
    for y in range(9):
        if(y%3==0):
            print("")            
        for x in range(9):
            if(x%3 == 0):
                print(" ", end='')
            print(solver.Value(nodes[x][y]), end='')            
        print("")
    

In [237]:
print_solution(solver,nodes)


 315 264 897
 769 583 412
 824 197 563

 953 471 286
 248 956 371
 176 328 945

 587 642 139
 691 835 724
 432 719 658
